In [94]:
# Warning: Normally its '  but in the text is ’
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)
    # general
    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)
    return phrase

def sentence2Vec(dataset,model,wordDim,maxLength):
    sentenceVec = []
    for sentence in dataset:
        wordVec = []
        for vocab in sentence:
            if vocab == marked_token:
                wordVec.append(np.ones(wordDim))
            else:
                try:
                    vec =  model[vocab]
                    wordVec.append(vec)
                except :
                    wordVec.append(np.zeros(wordDim))
        sentenceVec.append(wordVec)

    sentenceVec = keras.preprocessing.sequence.pad_sequences(sentenceVec, maxlen=maxLength, dtype='float', padding='post', truncating='post', value=0.0)
    return sentenceVec 


# High frequency vocab dictionary 
def getHighFrequencyVocab(frequentVocabLimit,mergeList):
    fdist1 = FreqDist(mergeList)
    frequentVocab = fdist1.most_common(frequentVocabLimit)
    frequentDic = {}
    for vocab in frequentVocab:
        frequentDic[vocab[0]] = True
    return frequentDic


def generateMaskedSentence(processedSentence,frequentDic):
    for sentence in processedSentence:
        for i in range(len(sentence)):
            if sentence[i] not in frequentDic.keys():
                sentence[i] = marked_token
    return processedSentence

def generateIndexForDataset(dataSize):
    trainIndexList = []
    data_index = []

    for i in range(dataSize-1):
        x = randint(0,dataSize-1)
        y = randint(0,dataSize-1)
        z = i+1 
        while(x == y or x==i or x==z or y == i or y==z ):
            x = randint(0,dataSize-1)
            y = randint(0,dataSize-1)
        index = ([[x,0],[y,0],[z,1]])
        random.shuffle(index)
        trainIndexList.append([i,index])   

    for row in trainIndexList:
        train_x = row[0]
        y_index = row[1]
        y_output = []
        x_masked = []
        for index in y_index:
            y_output.append(index[1])
            x_masked.append(index[0])
        data_index.append([train_x,x_masked,y_output])

    random.shuffle(data_index)

    return data_index


def loadEmbeddingModel(embeddingPath,word2VecFile,gloveFile,convertGloveToWordVec):
    word2vec_output_file = embeddingPath + word2VecFile
    glove_input_file = embeddingPath + gloveFile

    if convertGloveToWordVec:
        glove2word2vec(glove_input_file, word2vec_output_file)

    # load the Stanford GloVe model
    model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
    return model



def loadFile(textPath,orgFile):
    with open(textPath +  orgFile ,encoding="utf-8") as fp:
        lines = fp.readlines()
    # Remove the empty line 
    lines = [e for e in lines if e not in {'\n'}]
    
    return lines
    
def preprocessing(lines): 
    newLine = []
    for line in lines:
        # Decapitalize: Conver to lower case first. 
        line = line.lower()
        # Delete url 
        line = re.sub(r"http\S+", "link", line)
        line = re.sub(r"\S+html", "link", line)
        line = re.sub(r"\S+.com$", "link", line)
        line = re.sub(r"\S+.jpg$", "photo", line)
        line = decontracted(line)

        '''
        ignore:  * { } \  < > 
        Splite based on : ! . ,  &  # ' $ 
        line = re.findall(r"[\w']+|[().,:!?;'$&]", line)
        '''

        line = re.findall(r"[\w']+|[().,:!?;'$&]", line)

        newLine.append(line)
    
    mergeList = []
    for i in newLine:
        mergeList += i   
    return newLine, mergeList

## Read file & Prepocessing

In [2]:
import pickle  
from nltk import FreqDist
import re
import random
from random import randint
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
import numpy as np 

embeddingPath = "Embedding/"
convertGloveToWordVec = False  # The first time use Glove need to convert to Word2Vec format
gloveFile = 'glove.840B.300d.txt'  
word2VecFile =  'word2vec.6B.50d.txt'
wordDim = 50

textPath = "text/"
orgFile = "all.txt"
processedFile = "process.txt"
maskedFile = "masked.txt"

frequentVocabLimit = 2000
marked_token = "MASKED_TOKEN"
trainingSize = 0
maxLength = 50
dataSize = 10

In [4]:
# Load the data, remove the empty line
sentenceList = loadFile(textPath,orgFile)
# Remove unwanted feature, proprocessing, return wordList
processedLine, vocabList = preprocessing(sentenceList)

dataSize = len(processedLine)
# Get the high frequency vocabulary 
frequenentVocabDic =  getHighFrequencyVocab(frequentVocabLimit,vocabList)

# Save processed data
with open(textPath +  processedFile, "wb") as internal_filename:
    pickle.dump(processedLine, internal_filename)

# Generate the masked dataset

maskedLine =  generateMaskedSentence(processedLine,frequenentVocabDic)

# Save masked data
with open(textPath +  maskedFile, "wb") as internal_filename:
    pickle.dump(maskedLine, internal_filename)
    
# Generate the index for the dataset 
dataIndex = generateIndexForDataset(dataSize)

# Read the data 
with open(textPath + processedFile, "rb") as input_file:
    processedLine = pickle.load(input_file)
with open(textPath + maskedFile , "rb") as input_file:
    maskedSentence = pickle.load(input_file)
    
model = loadEmbeddingModel(embeddingPath,word2VecFile,gloveFile,convertGloveToWordVec)


In [95]:
sentenceVec =  sentence2Vec(processedLine,model,wordDim,maxLength)
maskedSentenceVec = sentence2Vec(processedLine,model,wordDim,maxLength)

In [101]:
# Generate input_X, input_masked_X, input_Y
input_X = []
input_masked_first = []
input_masked_second = []
input_masked_third = []
input_Y = []
for index in dataIndex:
    input_X.append(sentenceVec[index[0]])
    input_Y.append(index[2])
    input_masked_first.append(maskedSentenceVec[index[1][0]])
    input_masked_second.append(maskedSentenceVec[index[1][1]])
    input_masked_third.append(maskedSentenceVec[index[1][2]])
    
input_X = np.array(input_X)
input_masked_first = np.array(input_masked_first)
input_masked_second = np.array(input_masked_second)
input_masked_third = np.array(input_masked_third)
input_Y = np.array(input_Y)

## Modeling  

In [3]:
import keras
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Bidirectional,Input, Dropout, Flatten, concatenate, dot, GaussianDropout, Activation, GRU
from keras.regularizers import l1, l2
import keras.backend as kb

/Users/chenxi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [102]:
num_neurons = [100,100]
kernel_reg = 0.01
merge_mode = 'concat'
batch_size = 258
epochs = 1

In [103]:
def buildModel(input_X,kernel_reg,num_neurons,merge_mode):
    org_input = Input(shape=(input_X.shape[1],input_X.shape[2]), name = 'org_input')
    org_input_1 = Bidirectional(GRU(num_neurons[0],return_sequences= True,kernel_regularizer=l2(kernel_reg)), merge_mode= merge_mode,name='X_input_first')(org_input)
    org_input_2 = Bidirectional(GRU(num_neurons[1],kernel_regularizer=l2(kernel_reg)), merge_mode= merge_mode,name='X_input_second')(org_input_1)

    # Create shared layer for masked sentence 
    shared_first = Bidirectional(GRU(num_neurons[0],return_sequences= True,kernel_regularizer=l2(kernel_reg)), merge_mode= merge_mode,name='X_input_mask_1')
    shared_second = Bidirectional(GRU(num_neurons[1],kernel_regularizer=l2(kernel_reg)), merge_mode= merge_mode,name='X_input_mask_2')

    mask_input_first = Input(shape=(input_X.shape[1],input_X.shape[2]), name = 'mask_input_first')
    mask_input_first_1 = shared_first(mask_input_first)
    mask_input_first_2 = shared_second(mask_input_first_1)

    mask_input_second = Input(shape=(input_X.shape[1],input_X.shape[2]), name = 'mask_input_second')
    mask_input_second_1 = shared_first(mask_input_second)
    mask_input_second_2 = shared_second(mask_input_second_1)

    mask_input_third = Input(shape=(input_X.shape[1],input_X.shape[2]), name = 'mask_input_third')
    mask_input_third_1 = shared_first(mask_input_third)
    mask_input_third_2 = shared_second(mask_input_third_1)

    out1 = dot([org_input_2, mask_input_first_2], axes=1, name='output_1')
    out2 = dot([org_input_2, mask_input_second_2], axes=1, name='output_2')
    out3 = dot([org_input_2, mask_input_third_2], axes=1, name='output_3')

    concat = concatenate([out1, out2, out3], name='concat')

    output = Activation('softmax')(concat)

    model = Model(inputs=[org_input,mask_input_first,mask_input_second,mask_input_third], outputs=output)

    return model


In [106]:
model = buildModel(input_X,kernel_reg,num_neurons,merge_mode)
model.compile( loss='mse', optimizer='adam')
model.summary()

org_dim_1 dimension is:(None, 50, 200), org_dim_2 dimension is: (None, 200)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
org_input (InputLayer)          (None, 50, 50)       0                                            
__________________________________________________________________________________________________
mask_input_first (InputLayer)   (None, 50, 50)       0                                            
__________________________________________________________________________________________________
mask_input_second (InputLayer)  (None, 50, 50)       0                                            
__________________________________________________________________________________________________
mask_input_third (InputLayer)   (None, 50, 50)       0                                            
_________________________________

In [107]:
model.fit([input_X,input_masked_first,input_masked_second,input_masked_third],input_Y,epochs=epochs,batch_size=batch_size)

Epoch 1/1
12793/12793 [==============================] - 202s 16ms/step - loss: 14.6022


## Extract hidden layer 

In [108]:
layer_name = ['X_input_second','X_input_mask_2']

input_predict = [input_X[:10],input_X[:10],input_X[:10],input_X[:10]]

hidden_layer = []
for layer_name in layerName:
    intermediate_layer_model = Model(inputs=model.input,
                                     outputs=model.get_layer(layer_name).output)
    intermediate_output = intermediate_layer_model.predict(input_predict)
    hidden_layer.append(intermediate_output)

## SentEval